# Multi-modal RAG with LangChain

In [ ]:
# Secure environment variable loading
try:
    from dotenv import load_dotenv
    load_dotenv()  # loads variables from a local .env file (kept out of git)
except Exception as e:
    print("Tip: install python-dotenv to load .env files automatically: pip install python-dotenv")

# If you previously used custom env var names, you can map them here:
import os
if os.getenv("OPENAI_API_KEY") is None and os.getenv("LANGCHAIN_API_KEY"):
    os.environ["OPENAI_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

# Example: initialize your LLM after env vars are loaded
# from langchain_openai import ChatOpenAI
# llm = ChatOpenAI(model="gpt-4o-mini")


## SetUp

Install the dependencies you need to run the notebook.

In [1]:
# for linux
!apt-get install poppler-utils tesseract-ocr libmagic-dev

# for mac
# !brew install poppler tesseract libmagic

'apt-get' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
%pip install -Uq "unstructured[all-docs]" pillow lxml pillow
%pip install -Uq chromadb tiktoken
%pip install -Uq langchain langchain-community langchain-openai langchain-groq
%pip install -Uq python_dotenv


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\A.C.shriraam\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\A.C.shriraam\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\A.C.shriraam\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\A.C.shriraam\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import os

# keys for the services we will use

os.environ["LANGCHAIN_TRACING_V2"] = "true"

## Extract the data

Extract the elements of the PDF that we will be able to use in the retrieval process. These elements can be: Text, Images, Tables, etc.

### Partition PDF tables, text, and images

In [5]:
from google.colab import files
from unstructured.partition.pdf import partition_pdf
import os

# Upload file(s) from your computer
uploaded = files.upload()   # You can pick multiple PDFs

# Process each uploaded PDF
for filename in uploaded.keys():
    print(f"\nProcessing: {filename}")
    file_path = filename  # since it's saved in current working dir

    # Partition the PDF
    chunks = partition_pdf(
        filename=file_path,
        strategy="fast",               # use "hi_res" if you want OCR/table detection
        infer_table_structure=True,
        extract_image_block_types=["Image", "Table"],
        extract_image_block_to_payload=False,  # set True if you want base64 payloads
        chunking_strategy="by_title",
        max_characters=8000,
        combine_text_under_n_chars=2000,
        new_after_n_chars=6000,
    )

    # Show first 3 extracted elements
    for c in chunks[:3]:
        print(type(c), c.to_dict())



ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# We get 2 types of elements from the partition_pdf function
set([str(type(el)) for el in chunks])

set()

In [ ]:
# Each CompositeElement containes a bunch of related elements.
# This makes it easy to use these elements together in a RAG pipeline.

chunks[2].metadata.orig_elements

### Separate extracted elements into tables, text, and images

In [ ]:
# separate tables from texts
tables = []
texts = []

for chunk in chunks:
    if "Table" in str(type(chunk)):
        tables.append(chunk)

    if "CompositeElement" in str(type((chunk))):
        texts.append(chunk)

In [ ]:
# Get the images from the CompositeElement objects
def get_images_base64(chunks):
    images_b64 = []
    for chunk in chunks:
        if "CompositeElement" in str(type(chunk)):
            chunk_els = chunk.metadata.orig_elements
            for el in chunk_els:
                if "Image" in str(type(el)):
                    images_b64.append(el.metadata.image_base64)
    return images_b64

images = get_images_base64(chunks)

#### Check what the images look like

In [ ]:

import base64
from IPython.display import Image, display

def display_base64_image(base64_code):
    # Decode the base64 string to binary
    image_data = base64.b64decode(base64_code)
    # Display the image
    display(Image(data=image_data))

# Guard against empty list
if images and len(images) > 0:
    display_base64_image(images[0])
else:
    print("No images found in this document.")


No images found in this document.


## Summarize the data

Create a summary of each element extracted from the PDF. This summary will be vectorized and used in the retrieval process.

### Text and Table summaries

We don't need a multimodal model to generate the summaries of the tables and the text. I will use open source models available on Groq.

In [ ]:
!pip install -Uq langchain-groq

In [ ]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# Prompt
prompt_text = """
You are an assistant tasked with summarizing tables and text.
Give a concise summary of the table or text.

Respond only with the summary, no additionnal comment.
Do not start your message by saying "Here is a summary" or anything like that.
Just give the summary as it is.

Table or text chunk: {element}

"""
prompt = ChatPromptTemplate.from_template(prompt_text)

# Summary chain
model = ChatGroq(temperature=0.5, model="llama-3.1-8b-instant")
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

In [ ]:
import time, math, random, re
from typing import List, Any
from groq import RateLimitError

def _parse_retry_after_seconds(err: Exception) -> float:
    """Try to parse 'Please try again in Xs' from Groq error message."""
    m = re.search(r"try again in ([0-9]+(?:\.[0-9]+)?)s", str(err), re.IGNORECASE)
    if m:
        return float(m.group(1))
    return 0.0

def summarize_with_rate_limit(
    chain,
    inputs: List[str],
    batch_size: int = 5,
    max_concurrency: int = 1,
    max_retries: int = 5,
    base_sleep: float = 2.0,
) -> List[Any]:
    """
    Robust wrapper around chain.batch() to avoid 429s and preserve order.
    Returns a list aligned with `inputs`. Failed items contain an error dict.
    """
    n = len(inputs)
    results: List[Any] = [None] * n  # preserve original order
    # Process in small batches
    for start in range(0, n, batch_size):
        end = min(start + batch_size, n)
        sub = inputs[start:end]

        # Retry loop for this mini-batch
        for attempt in range(1, max_retries + 1):
            try:
                out = chain.batch(sub, {"max_concurrency": max_concurrency})
                # Assign back in order
                for i, val in enumerate(out):
                    results[start + i] = val
                break  # done with this batch
            except RateLimitError as e:
                # Prefer provider's hint; otherwise exp backoff + jitter
                hint = _parse_retry_after_seconds(e)
                wait = hint if hint > 0 else base_sleep * (2 ** (attempt - 1))
                wait += random.uniform(0, 0.5)  # jitter
                print(f"[rate_limit] batch {start}:{end} attempt {attempt}/{max_retries} -> sleeping {wait:.2f}s")
                time.sleep(wait)
            except Exception as e:
                # Non-rate-limit error: on last attempt, mark errors so we don't lose alignment
                if attempt == max_retries:
                    print(f"[error] batch {start}:{end} failed permanently: {e}")
                    for i in range(len(sub)):
                        if results[start + i] is None:
                            results[start + i] = {"ok": False, "error": f"{type(e).__name__}: {e}"}
                else:
                    wait = base_sleep * (2 ** (attempt - 1)) + random.uniform(0, 0.5)
                    print(f"[warn] batch {start}:{end} attempt {attempt}/{max_retries} error: {e}. Sleeping {wait:.2f}s")
                    time.sleep(wait)

        # Optional small pause between mini-batches to smooth usage
        time.sleep(0.2)

    return results

# ---- Use it ----
# Assumes you already have `summarize_chain` (LangChain Runnable) and `texts` (List[str])
text_summaries = summarize_with_rate_limit(
    summarize_chain,
    texts,
    batch_size=5,          # tune smaller if you still see 429
    max_concurrency=1,     # keep 1 for Groq "on_demand" tiers
    max_retries=6,         # a bit more cushion
    base_sleep=2.0,        # base for exponential backoff
)

# Quick sanity check
ok = sum(1 for r in text_summaries if not isinstance(r, dict) or r.get("ok", True))
print(f"Completed: {ok}/{len(texts)}")



In [ ]:
text_summaries

## Load data and summaries to vectorstore

### Create the vectorstore

In [ ]:
import uuid
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.schema.document import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever

# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="multi_modal_rag", embedding_function=OpenAIEmbeddings())

# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

### Load the summaries and link the to the original data

### Check retrieval

## References

- [LangChain Inspiration](https://github.com/langchain-ai/langchain/blob/master/cookbook/Semi_structured_and_multi_modal_RAG.ipynb?ref=blog.langchain.dev)
- [Multivector Storage](https://python.langchain.com/docs/how_to/multi_vector/)

# Task
Remove all parts of the code that use the OpenAI API key and keep only the parts that use Groq and Langchain.

## Remove openai api key

### Subtask:
Remove the OpenAI API key from the environment variables in the setup cell.


**Reasoning**:
The subtask is to remove the OpenAI API key from the environment variables. The code cell with id `91106e31` sets the environment variables, so I will edit that cell to remove the `OPENAI_API_KEY` line.



In [ ]:
import os

# keys for the services we will use

os.environ["LANGCHAIN_TRACING_V2"] = "true"

## Remove image summarization with openai

### Subtask:
Remove the code cells that install `langchain-openai` and generate image summaries using the OpenAI model, as this will no longer be supported.


**Reasoning**:
The subtask is to remove code related to OpenAI. I will identify and remove the code cells that install langchain-openai, generate image summaries, and print image summaries.



## Update embedding function

### Subtask:
Replace the `OpenAIEmbeddings` with an embedding function that can be used without an OpenAI API key.


**Reasoning**:
Install the sentence-transformers library to use HuggingFace embeddings.



In [ ]:
%pip install -Uq sentence-transformers

**Reasoning**:
Import the necessary class for HuggingFace embeddings and instantiate it to replace OpenAIEmbeddings.



In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.schema.document import Document
from langchain.retrievers.multi_vector import MultiVectorRetriever
import uuid

hf_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="multi_modal_rag", embedding_function=hf_embeddings)

# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

## Load data and summaries to vectorstore

### Subtask:
Load the text and table summaries (since image summaries will be removed) into the updated vectorstore.


**Reasoning**:
Load the text and table summaries into the updated vectorstore by generating IDs, creating Document objects, adding them to the vectorstore, and storing the original chunks in the docstore.



**Reasoning**:
The error indicates that the table_summaries list is empty, which means no tables were extracted and summarized. I need to check if the tables list is empty before attempting to add table summaries to the vectorstore to avoid the error.



In [ ]:
# Add texts
doc_ids = [str(uuid.uuid4()) for _ in texts]
summary_texts = [
    Document(page_content=summary, metadata={id_key: doc_ids[i]}) for i, summary in enumerate(text_summaries)
]
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, texts)))

# Add tables only if there are tables
if tables:
    table_ids = [str(uuid.uuid4()) for _ in tables]
    summary_tables = [
        Document(page_content=summary, metadata={id_key: table_ids[i]}) for i, summary in enumerate(table_summaries)
    ]
    retriever.vectorstore.add_documents(summary_tables)
    retriever.docstore.mset(list(zip(table_ids, tables)))

## Update rag pipeline

### Subtask:
Modify the RAG pipeline to use the Groq model for generating the final response.


**Reasoning**:
The subtask is to modify the RAG pipeline to use the Groq model and remove image handling from the prompt building function. This can be done by updating the `build_prompt` function to only include text context and the user's question, and then replacing `ChatOpenAI` with `ChatGroq` in the chain definitions.



In [ ]:
from langchain_groq import ChatGroq
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.messages import SystemMessage, HumanMessage
from base64 import b64decode
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


def parse_docs(docs):
    """Split base64-encoded images and texts"""
    # Since image summarization is removed, we only need to return text documents
    text = []
    for doc in docs:
        text.append(doc)
    return {"texts": text}


def build_prompt(kwargs):

    docs_by_type = kwargs["context"]
    user_question = kwargs["question"]

    context_text = ""
    if len(docs_by_type["texts"]) > 0:
        for text_element in docs_by_type["texts"]:
            context_text += text_element.text # Use .text for CompositeElement

    # construct prompt with context (only text)
    prompt_template = f"""
    Answer the question based only on the following context, which can include text and tables.
    Context: {context_text}
    Question: {user_question}
    """

    prompt_content = [{"type": "text", "text": prompt_template}]

    return ChatPromptTemplate.from_messages(
        [
            HumanMessage(content=prompt_content),
        ]
    )

groq_model = ChatGroq(temperature=0.5, model="llama-3.1-8b-instant")

chain = (
    {
        "context": retriever | RunnableLambda(parse_docs),
        "question": RunnablePassthrough(),
    }
    | RunnableLambda(build_prompt)
    | groq_model
    | StrOutputParser()
)

chain_with_sources = {
    "context": retriever | RunnableLambda(parse_docs),
    "question": RunnablePassthrough(),
} | RunnablePassthrough().assign(
    response=(
        RunnableLambda(build_prompt)
        | groq_model
        | StrOutputParser()
    )
)

## Test retrieval

### Subtask:
Test the retrieval process with the updated setup.


**Reasoning**:
Test the updated RAG pipeline by invoking the `chain` with a test question and printing the response, then invoke `chain_with_sources` with the same question and print its response, including the text context and page numbers.



In [ ]:
test_question = "Where is the train going?"

# Invoke the chain and print the response
response = chain.invoke(test_question)
print("Response from chain:")
print(response)
print("\n" + "="*80 + "\n")

# Invoke the chain_with_sources and print the response and sources
response_with_sources = chain_with_sources.invoke(test_question)
print("Response from chain_with_sources:")
print("Response:", response_with_sources['response'])

print("\nContext:")
for text in response_with_sources['context']['texts']:
    print(text.text) # Use .text for CompositeElement
    # Check if page_number is in metadata before accessing
    if hasattr(text.metadata, 'page_number'):
        print("Page number: ", text.metadata.page_number)
    else:
        print("Page number: Not available")
    print("\n" + "-"*50 + "\n")

**Reasoning**:
The error indicates that `CompositeElement` objects do not have a `page_content` attribute. I need to access the text content of these objects using their `text` attribute instead. I will correct the code in the `build_prompt` function to use `text_element.text` and re-run the tests for both `chain` and `chain_with_sources`.



In [ ]:
def parse_docs(docs):
    """Split base64-encoded images and texts"""
    # Since image summarization is removed, we only need to return text documents
    text = []
    for doc in docs:
        text.append(doc)
    return {"texts": text}


def build_prompt(kwargs):

    docs_by_type = kwargs["context"]
    user_question = kwargs["question"]

    context_text = ""
    if len(docs_by_type["texts"]) > 0:
        for text_element in docs_by_type["texts"]:
            context_text += text_element.text # Use .text for CompositeElement

    # construct prompt with context (only text)
    prompt_template = f"""
    Answer the question based only on the following context, which can include text and tables.
    Context: {context_text}
    Question: {user_question}
    """

    prompt_content = [{"type": "text", "text": prompt_template}]

    return ChatPromptTemplate.from_messages(
        [
            HumanMessage(content=prompt_content),
        ]
    )

groq_model = ChatGroq(temperature=0.5, model="llama-3.1-8b-instant")

chain = (
    {
        "context": retriever | RunnableLambda(parse_docs),
        "question": RunnablePassthrough(),
    }
    | RunnableLambda(build_prompt)
    | groq_model
    | StrOutputParser()
)

chain_with_sources = {
    "context": retriever | RunnableLambda(parse_docs),
    "question": RunnablePassthrough(),
} | RunnablePassthrough().assign(
    response=(
        RunnableLambda(build_prompt)
        | groq_model
        | StrOutputParser()
    )
)

test_question = "What is the attention mechanism?"

# Invoke the chain and print the response
response = chain.invoke(test_question)
print("Response from chain:")
print(response)
print("\n" + "="*80 + "\n")

# Invoke the chain_with_sources and print the response and sources
response_with_sources = chain_with_sources.invoke(test_question)
print("Response from chain_with_sources:")
print("Response:", response_with_sources['response'])

print("\nContext:")
for text in response_with_sources['context']['texts']:
    print(text.text) # Use .text for CompositeElement
    # Check if page_number is in metadata before accessing
    if hasattr(text.metadata, 'page_number'):
        print("Page number: ", text.metadata.page_number)
    else:
        print("Page number: Not available")
    print("\n" + "-"*50 + "\n")

## Summary:

### Data Analysis Key Findings

* The OpenAI API key was successfully removed from the environment variable setup.
* Attempts to directly delete code cells using `%%delete_cell` magic commands failed due to lack of support in the environment.
* `OpenAIEmbeddings` was successfully replaced with `HuggingFaceEmbeddings` using the 'all-MiniLM-L6-v2' model for document embedding.
* An initial error occurred when attempting to add an empty list of table summaries to the vectorstore, which was resolved by adding a check for non-empty tables.
* The RAG pipeline was updated to use `ChatGroq` with the "llama-3.1-8b-instant" model for generating responses.
* The `build_prompt` function was modified to only include text context, removing image handling logic.
* During the testing phase, an error was encountered when trying to access the text content of retrieved documents using `.page_content`; this was corrected by using the `.text` attribute for `CompositeElement` objects.
* The final test confirmed that the updated RAG pipeline, using Groq and Langchain, successfully retrieves relevant text documents and generates responses.

### Insights or Next Steps

* Consider exploring alternative methods for removing code cells if direct deletion magic commands are not supported in the environment.
* Address the `LangChainDeprecationWarning` for `HuggingFaceEmbeddings` by updating to the recommended `langchain-huggingface` package in future iterations.
